<a href="https://colab.research.google.com/github/nakib103/tensorflow/blob/master/eager_execution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import tensorflow as tf
import cProfile

print(tf.__version__)

In [3]:
tf.executing_eagerly()

True

In [5]:
x = [[2.]]
print(type(x))

<class 'list'>


In [8]:
# tf.Tensor objects reference concrete values instead of symbolic handles to nodes in a computational graph
m = tf.matmul(x, x)
# Since there isn't a computational graph to build and run later in a session, it's easy to inspect results using print() or a debugger. 
print("Matric multiplication result {}".format(m))

Matric multiplication result [[4.]]


In [9]:
a = tf.constant([[1., 3.],
                 [2., 4.]])

b = tf.constant([[3., 1.],
                 [5., 2.]])

print(a * b)

tf.Tensor(
[[ 3.  3.]
 [10.  8.]], shape=(2, 2), dtype=float32)


In [12]:
# Eager execution works nicely with NumPy. NumPy operations accept tf.Tensor arguments
import numpy as np
c = np.multiply(a, b)
print(c)

print(a.numpy())

[[ 3.  3.]
 [10.  8.]]
[[1. 3.]
 [2. 4.]]


In [ ]:
# functionality of the host language is available while your model is executing.

def iffneff(num):
  counter = tf.constant(0)

  max = tf.convert_to_tensor(num)
  for idx in range(1, max.numpy()+1):
    if(idx % 3 == 0):
      print("iffff")
    else:
      print("effff")

  counter += 1

iffneff(20)

In [20]:
# During eager execution, use tf.GradientTape to trace operations for computing gradients later
w = tf.Variable([3.])
with tf.GradientTape() as tape:
  mul = w * w * w

grad = tape.gradient(mul, w)
print(grad)

tf.Tensor([27.], shape=(1,), dtype=float32)


In [21]:
# Even without training, call the model and inspect the output in eager execution
(mnist_images, mnist_labels), _ = tf.keras.datasets.mnist.load_data()

dataset = tf.data.Dataset.from_tensor_slices(
  (tf.cast(mnist_images[...,tf.newaxis]/255, tf.float32),
   tf.cast(mnist_labels,tf.int64)))
dataset = dataset.shuffle(1000).batch(32)

mnist_model = tf.keras.Sequential([
  tf.keras.layers.Conv2D(16,[3,3], activation='relu',
                         input_shape=(None, None, 1)),
  tf.keras.layers.Conv2D(16,[3,3], activation='relu'),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(10)
])

for images,labels in dataset.take(1):
  print("Logits: ", mnist_model(images[0:1]).numpy())

11493376/11490434 [==============================] - 0s 0us/step
Logits:  [[ 0.03187805  0.04470838 -0.02943167 -0.03389054 -0.03579982  0.04431757
   0.03672335 -0.03304952 -0.01135808 -0.03420841]]


In [22]:
# training loop implemented with eager
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

loss_history = []

def train_step(images, labels):
  with tf.GradientTape() as tape:
    logits = mnist_model(images, training=True)
    
    # Add asserts to check the shape of the output.
    tf.debugging.assert_equal(logits.shape, (32, 10))
    
    loss_value = loss_object(labels, logits)

  loss_history.append(loss_value.numpy().mean())
  grads = tape.gradient(loss_value, mnist_model.trainable_variables)
  optimizer.apply_gradients(zip(grads, mnist_model.trainable_variables))

def train(epochs):
  for epoch in range(epochs):
    for (batch, (images, labels)) in enumerate(dataset):
      train_step(images, labels)
    print ('Epoch {} finished'.format(epoch))

train(epochs = 3)

Epoch 0 finished
Epoch 1 finished
Epoch 2 finished


In [26]:
# tf.Variable objects store mutable tf.Tensor-like values accessed during training to make automatic differentiation easier
class Linear(tf.keras.Model):
  def __init__(self):
    super(Linear, self).__init__()            # calls super class init function
    self.W = tf.Variable(5., name="weight")
    self.B = tf.Variable(10., name="bias")

  def call(self, inputs):                     # forward pass is defined in call method
    return inputs * self.W + self.B

def loss(model, inputs, targets):
  error = model(inputs) - targets
  return tf.reduce_mean(tf.square(error))

def grad(model, inputs, targets):
  with tf.GradientTape() as tape:
    loss_value = loss(model, inputs, targets)
  return tape.gradient(loss_value, [model.W, model.B])

NUM_EXAMPLES = 2000
training_inputs = tf.random.normal([NUM_EXAMPLES])
noise = tf.random.normal([NUM_EXAMPLES])
training_outputs = training_inputs * 3 + 2 + noise

model = Linear()
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

print("Initial loss: {:.3f}".format(loss(model, training_inputs, training_outputs)))

steps = 300
for i in range(steps):
  grads = grad(model, training_inputs, training_outputs)
  optimizer.apply_gradients(zip(grads, [model.W, model.B]))
  if i % 20 == 0:
    print("Loss at step {:03d}: {:.3f}".format(i, loss(model, training_inputs, training_outputs)))

Initial loss: 69.235
Loss at step 000: 66.534
Loss at step 020: 30.224
Loss at step 040: 14.034
Loss at step 060: 6.814
Loss at step 080: 3.595
Loss at step 100: 2.159
Loss at step 120: 1.519
Loss at step 140: 1.233
Loss at step 160: 1.106
Loss at step 180: 1.049
Loss at step 200: 1.023
Loss at step 220: 1.012
Loss at step 240: 1.007
Loss at step 260: 1.005
Loss at step 280: 1.004


In [ ]:
# object-based saving
# object-oriented metrics
# summaries and tensorboard

# dynamic model
# custom gradients
# performance
# benchmarks
# work with functions